# EOF


## Using package


In [1]:
using Plots
using NCDatasets
using Statistics
using LinearAlgebra
using MultivariateStats


In [2]:
using LinearAlgebra

function EmpOrthFunc(data)
    # Calculate Covariance Matrix
    CovMat = (data * data') / size(data, 2)

    eigvals = eigvals(CovMat)
    eigvecs = eigvecs(CovMat)

    # Explained Variance Ratio
    ExpVar = eigvals ./ sum(eigvals)

    # Sort eigenvalues and eigenvectors together by decreasing explained variance
    sorting = reverse(argsort(ExpVar))
    ExpVar = ExpVar[sorting]
    eigvecs = eigvecs[:, sorting]

    # Adjust sign of EOFs if necessary
    if eigvecs[length(data) // 2, 1] < 0
        EOF = -eigvecs
    else
        EOF = eigvecs
    end

    # Normalize eigenvectors (optional, based on your use case)
    # EOF = (EOF .- mean(EOF, dims=1)) ./ std(EOF, dims=1)

    # Project data onto principal components
    PC = eigvecs' * data
    return ExpVar, EOF, PC
end

EmpOrthFunc (generic function with 1 method)

## Load data


In [3]:
filename = "/work/b11209013/2024_Research/MPAS/merged_data/CNTL/q1.nc"

q1_file = Dataset(filename, "r");

dims::Dict = Dict(
  "lat" => q1_file["lat"][:],
  "lev" => q1_file["lev"][:],
  "lon" => q1_file["lon"][:],
  "time" => q1_file["time"][:],
);

lat_cond = findall((dims["lat"] .>= -5) .& (dims["lat"] .<= 5));

dims["lat"] = dims["lat"][lat_cond];

q1 = q1_file["q1"][:, lat_cond, :, :]; # [lon, lat, lev, time]

llon, llat, llev, ltime = size(q1);

close(q1_file);



## Permute and reshape data


In [4]:
q1_pm = permutedims(q1, [3, 1, 2, 4]);
q1_rs = reshape(q1_pm, (llev, llon*llat*ltime));


## EOF/PCA on data


In [5]:
# scaling data
q1_scaled = (q1_rs .- mean(q1_rs, dims=1))./std(q1_rs, dims=1)

n_components=2

pca = fit(PCA, q1_scaled, maxoutdim=n_components)

pcs = transform(pca, q1_scaled)
eofs = (eigvecs(pca));

In [6]:
CovMat = (q1_rs * q1_rs') / size(q1_rs, 2)

eigvals = eigvals(CovMat)
eigvecs = eigvecs(CovMat)

LoadError: cannot assign a value to imported variable LinearAlgebra.eigvals from module Main

In [ ]:
plot(eofs[:, 1], dims["lev"])

LoadError: type Array has no field arr